In [2]:
from tensorflow.python.keras.models import Input, Model
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, AveragePooling2D, Dense, Activation, Flatten, Reshape, Add, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np

def relu6(inputs):
    '''
        Performs the ReLU6 activation function for the bottleneck stage of the MobileNet V2
        Inputs:
            inputs: the layer with the inputs for the activation function
        Return:
            Min value between the value of the regular ReLU function and 6
    '''
    
    return K.relu(inputs,max_value=6)

def bottleneck(inputs, t, alpha, num_filters, kernel_sz=(3,3),stride=(1,1),pad='same',residual=False,dropout=False,dropout_perc=0.1):    
    '''
        Performs the bottleneck stage of the MobileNet V2
        Inputs:
            inputs: the layer with the inputs
            t: the value used to increase the number of filters of the expansion stage
            alpha: width multiplier that controls the number of filters of the output tensor
            num_filters: number of filters of the output tensor
            kernel_sz = kernel size of the filter
            stride: stride of the kernel
            pad: padding of the filter
            residual: parameter that determine the sum of the input and output of the bottleneck stage
            dropout: determine if dropout will be performed 
            dropout_perc: percentage of neurons that will be set to zero
        Return:
            x: the result of the bottleneck stage
    '''    
    
    # Get the index of the input 4D tensor that represents the number of channels of the image
    # -1 can also represent the last element of the tensor
    channel_idx = 1 if K.image_data_format == 'channels_first' else -1
    
    # Number of filters for the expansion convolution
    num_filters_exp = K.int_shape(inputs)[channel_idx] * t    
    
    # Number of filters of the projection convolution
    num_filters_proj = int(num_filters * alpha)
    
    # Expansion layer
    x = Conv2D(filters=num_filters_exp,kernel_size=(1,1),strides=(1,1),padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=kernel_sz,strides=stride,depth_multiplier=1,padding=pad)(x)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    
    # Projection convolution
    x = Conv2D(filters=num_filters_proj,kernel_size=(1,1),strides=(1,1),padding=pad)(x)
    x = BatchNormalization()(x)
    
    if (residual == True):
        x = Add()([x,inputs])
        
    if (dropout == True):
      x = Dropout(dropout_perc)(x)
    
    return x

def depthwise_block(inputs,stride,kernel_sz=(3,3),pad='same'):
    '''
        Function that performs the depthwise convolution
        Inputs:
            inputs:    the input shape of the depthwise convolution
            kernel_sz: a tuple that indicates the size of the filtering kernel
            stride:    a tuple that indicates the strides of the kernel
        Return:
            x: the result of the depthwise convolution
    '''
        
    x = DepthwiseConv2D(kernel_size=kernel_sz,strides=stride,depth_multiplier=1,padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
        
    return x

def pointwise_block(inputs,num_filters,alpha,kernel_sz=(1,1),stride=(1,1),pad='same',dropout=False,dropout_perc=0.1):
    '''
        Function that performs the pointwise convolution
        Inputs:
            inputs:      the input shape of the depthwise convolution
            num_filters: number of filters to be used in the convolution
            kernel_sz:   a tuple that indicates the size of the filtering kernel
            stride:      a tuple that indicates the strides of the kernel
            dropout: determine if dropout will be performed 
            dropout_perc: percentage of neurons that will be set to zero            
        Return:
            x: the result of the pointwise convolution
    '''    
    
    # Number of filters based on width multiplier reported in the original paper
    n_fil = int(num_filters * alpha)    
    
    x = Conv2D(filters=n_fil,kernel_size=kernel_sz,padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    if (dropout == True):
      x = Dropout(dropout_perc)(x)
    
    return x


def MobileNetV2(input_shape, num_units, filters=32, kernel_sz=(3,3),stride=(2,2),alp=1,ro=1,dropout_perc=0.1):
    input_shape = (int(input_shape[0] * ro), int(input_shape[1] * ro), input_shape[2])
    
    inputs = Input(shape=input_shape)
    
    # Regular convolution
    x = Conv2D(filters=filters,kernel_size=kernel_sz,strides=stride)(inputs)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    x = Dropout(dropout_perc)(x)

    # First bottleneck convolution
    x = bottleneck(x,t=1,alpha=alp,num_filters=16,kernel_sz=(3,3),stride=(1,1),dropout=True,dropout_perc=dropout_perc)

    # Second bottleneck convolution (peformed 2 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=24,kernel_sz=(3,3),stride=(2,2),dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=24,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Third bottleneck convolution (peformed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Fourth bottleneck convolution (performed 4 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Fifth bottleneck convolution (performed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1))
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Sixth bottleneck convolution (performed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Seventh bottleneck convolution (performed 1 time)
    x = bottleneck(x,t=6,alpha=alp,num_filters=320,kernel_sz=(3,3),stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Eigth layer (regular convolution)
    x = Conv2D(filters=1280, kernel_size=(1,1), strides=(1,1), padding='same')(x)
    
    # Pooling layer
    # Pooling size correction due to the resolution multiplier parameter
    pool_size = int(np.round(7*ro))
    x = AveragePooling2D(padding='valid',pool_size=(pool_size,pool_size),strides=(1,1))(x)    
    
    x = Conv2D(filters=num_units,kernel_size=(1,1),strides=(1,1), padding='same')(x)
    
    output = Reshape((num_units,))(Activation(activation='softmax')(x))
    
    return Model(inputs,output)

In [3]:
alpha = 1
ro = 1
img_size = 224
target_sz = int(img_size * ro)
batch_sz = 32
epo = 100

In [4]:
import os

PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET = PROJECT+"-capstone"
REGION = "us-central1"

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [5]:
from google.cloud import storage
from collections import defaultdict
import os
import re

# Initialize the storage client
storage_client = storage.Client()

# Set bucket name from environment variable
bucket_name = os.environ["BUCKET"]
bucket = storage_client.bucket(bucket_name)

image_folder = "train"

# List all image files in the specified folder
blobs = bucket.list_blobs(prefix=image_folder)

image_urls = []
labels = []
images = []

# Function to extract label from the blob name
def extract_label(blob_name):
    # Example regex to extract label: 'train/category/image.jpg'
    match = re.search(r'train/([^/]+)/.*', blob_name)
    return match.group(1) if match else 'unknown'

# Collect image URLs and their labels
for blob in blobs:
    if blob.name.lower().endswith(('.png', '.jpg', '.jpeg')) and blob.name.lower().startswith('train/tomato'):
        image_urls.append(f"gs://{bucket_name}/{blob.name}")
        labels.append(extract_label(blob.name))

print(f"Found {len(image_urls)} images.")

# Count the number of images for each label
label_counts = defaultdict(int)
for label in labels:
    label_counts[label] += 1

# Print the count of images for each label
for label, count in label_counts.items():
    print(f"Label: {label}, Number of Images: {count}")


Found 18345 images.
Label: Tomato___Bacterial_spot, Number of Images: 1702
Label: Tomato___Early_blight, Number of Images: 1920
Label: Tomato___Late_blight, Number of Images: 1851
Label: Tomato___Leaf_Mold, Number of Images: 1882
Label: Tomato___Septoria_leaf_spot, Number of Images: 1745
Label: Tomato___Spider_mites Two-spotted_spider_mite, Number of Images: 1741
Label: Tomato___Target_Spot, Number of Images: 1827
Label: Tomato___Tomato_Yellow_Leaf_Curl_Virus, Number of Images: 1961
Label: Tomato___Tomato_mosaic_virus, Number of Images: 1790
Label: Tomato___healthy, Number of Images: 1926


In [ ]:
import tensorflow as tf

# Function to load and preprocess an image
def preprocess_image(image_url):
    image = tf.io.read_file(image_url)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize to [0, 1]
    return image

def load_data_from_gcs(image_urls):
    for url in image_urls:
        images.append(preprocess_image(url))
    return images


images = load_data_from_gcs(image_urls)

2024-06-21 08:49:01.187252: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 08:49:01.200989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 08:49:01.205331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
images[18344]

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.41908765, 0.42300922, 0.5053621 ],
        [0.49609843, 0.50001997, 0.5823729 ],
        [0.48267308, 0.48659465, 0.5689476 ],
        ...,
        [0.45522168, 0.44345698, 0.5258099 ],
        [0.29983875, 0.28807405, 0.370427  ],
        [0.47383353, 0.46206883, 0.54442173]],

       [[0.53881556, 0.5427371 , 0.62509006],
        [0.3688676 , 0.37278917, 0.4551421 ],
        [0.5855342 , 0.5894558 , 0.6718087 ],
        ...,
        [0.40980336, 0.39803866, 0.4803916 ],
        [0.31514546, 0.30338076, 0.3857337 ],
        [0.57611567, 0.56435096, 0.6467039 ]],

       [[0.5138455 , 0.5177671 , 0.60012   ],
        [0.69747895, 0.7014005 , 0.78375345],
        [0.609964  , 0.6138856 , 0.6962385 ],
        ...,
        [0.47349125, 0.46172655, 0.54407954],
        [0.49165627, 0.47989157, 0.56224453],
        [0.46864727, 0.45688257, 0.5392355 ]],

       ...,

       [[0.67132795, 0.6556417 , 0.71054363],
        [0.59

In [2]:
import tensorflow as tf

# Create a tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

# Define your preprocessing and augmentation logic
def preprocess(image, label):
    return image, label

# Apply preprocessing to the dataset
dataset = dataset.map(preprocess).batch(batch_sz).prefetch(tf.data.experimental.AUTOTUNE)

2024-06-21 09:37:57.956267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'images' is not defined

In [ ]:
model_v2 = MobileNetV2((img_size,img_size,3),num_units=2,alp=alpha,ro=ro,dropout_perc=0.2)

optimizer = Adam(lr=0.001)

model_v2.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

model_check_point_v2 = ModelCheckpoint(filepath='../model/model_mb_v2.hdf5',monitor='val_loss',verbose=1,save_best_only=False,save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.7,patience=10,verbose=1,min_lr=0.0001)

step_size_train = train_generator.n/train_generator.batch_size
step_size_test = test_generator.n/test_generator.batch_size

history_train_v2 = model_v2.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=epo, validation_data=test_generator,validation_steps=step_size_test,
                    workers=8,
                    use_multiprocessing=False,
                    callbacks=[reduce_lr])

model_v2.save('../model/model_mb_v2.hdf5')

In [8]:
import os
os.getcwd()

'/home/jupyter/asl-ml-project/dataset'

In [ ]:
import pandas as pd
hist_df_v1 = pd.DataFrame(history_train_v1.history)

In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/history_mobilenet_v1.json',mode='w') as f:
  hist_df_v1.to_json(f)

In [ ]:
from keras.models import load_model
mb_v1 = load_model('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/model_mb_v1_final.hdf5')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history_train_v1.history['loss'],c='r',label='Training loss')
plt.plot(history_train_v1.history['val_loss'],c='b',label='Validation loss')
plt.title('Loss MobileNet V1')
plt.legend()
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/loss_mbnet_v1.jpg')

In [ ]:
plt.plot(history_train_v1.history['acc'],c='r',label='Training accuracy')
plt.plot(history_train_v1.history['val_acc'],c='b',label='Validation accuracy')
plt.title('Accuracy MobileNet V1')
plt.legend()
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/accuracy_mbnet_v1.jpg')

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255)
test_generator = image_generator.flow_from_directory('/content/gdrive/My Drive/Imagens/Airplane models/Teste',
                                                     target_size=(target_sz,target_sz),
                                                     color_mode='rgb',
                                                     batch_size=batch_sz,
                                                     class_mode='categorical',
                                                     shuffle=False)

Found 822 images belonging to 2 classes.


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = mb_v1.predict_generator(test_generator,workers=8,steps=step_size_test)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_generator.classes
target_names = ['A320','B737']
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred,target_names=target_names))

In [ ]:
from keras.preprocessing import image
img = image.load_img('/content/gdrive/My Drive/Imagens/Airplane models/Validação/B737/boeing_737_700_566386.jpg',target_size=(target_sz,target_sz))
x = image.img_to_array(img)/255.

x = x.reshape((-1,target_sz,target_sz,3))

pred = mb_v1.predict(x)

predicted_class_index = np.argmax(pred,axis=1)

labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
prediction = [labels[k] for k in predicted_class_index]

plt.figure()
plt.imshow(img)
plt.title('Airplane model: '+str(prediction))
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/B737_4.jpg')
pred